<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, uncomment and run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter

You should consider upgrading via the 'c:\users\jake\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [ ]:
! pip uninstall twitter
! pip install python-twitter

In [9]:
import twitter, re, datetime, pandas as pd

# your keys go here:
twitter_keys = {
    'consumer_key':        'E4trOi93W5C32wF5LQQYYgn5r',
    'consumer_secret':     'wwDbpb2PBLGxY59mBelqaE2aZspA8qiXEwNix6jSVd62LVXIYq',
    'access_token_key':    '733259692130979840-ODfJ6ViqSr5QrkrAWEDL0YbHPJHIXfo',
    'access_token_secret': 'G1bTC5NzgMmIfSmgpd58YWpA21FgkZ8V9twyRVOfYYEfq'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [10]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [11]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [15]:
# A:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [13]:
print(sanders[0])

{'tweet_id': 1268356809695297537, 'handle': 'Bernie Sanders', 'retweet_count': 968, 'text': "This is a rather extraordinary statement by James Mattis, retired U.S. Marine Corps General, and Trump's former Sec… https://t.co/bUQAcfZG9N", 'mined_at': datetime.datetime(2020, 6, 3, 18, 53, 47, 829249), 'created_at': 'Thu Jun 04 01:40:03 +0000 2020'}


In [16]:
print(donald[0])

{'tweet_id': 1268358305170296833, 'handle': 'Donald J. Trump', 'retweet_count': 6283, 'text': 'I feel more and more confident that our economy is in the early stages of coming back very strong. Not everyone agr… https://t.co/n26pVDAGOq', 'mined_at': datetime.datetime(2020, 6, 3, 18, 57, 14, 518598), 'created_at': 'Thu Jun 04 01:45:59 +0000 2020'}


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [24]:
# A:
trump_df = pd.DataFrame(donald)
sanders_df = pd.DataFrame(sanders)

In [25]:
trump_df.head()
sanders_df.head()

,tweet_id,handle,retweet_count,text,mined_at,created_at
0,1268356809695297537,Bernie Sanders,1109,This is a rather extraordinary statement by Ja...,2020-06-03 18:57:14.123960,Thu Jun 04 01:40:03 +0000 2020
1,1268301551648616450,Bernie Sanders,2830,“There’s no doubt that this is America’s momen...,2020-06-03 18:57:14.123960,Wed Jun 03 22:00:28 +0000 2020
2,1268281873932914694,Bernie Sanders,749,"Congratulations to @amandaforpa, @NikilSaval, ...",2020-06-03 18:57:14.216775,Wed Jun 03 20:42:16 +0000 2020
3,1268213580396191750,Bernie Sanders,13636,I don't often agree with Defense Secretary Esp...,2020-06-03 18:57:14.216775,Wed Jun 03 16:10:54 +0000 2020
4,1267935000114184195,Bernie Sanders,1278,This is an unprecedented moment for our countr...,2020-06-03 18:57:14.297568,Tue Jun 02 21:43:55 +0000 2020


In [27]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.tail()

,tweet_id,handle,retweet_count,text,mined_at,created_at
5,1267894979827040259,Bernie Sanders,27761,Those in power who want us to focus on looting...,2020-06-03 18:57:14.297568,Tue Jun 02 19:04:54 +0000 2020
6,1267624337038852097,Bernie Sanders,218152,Trump just had peaceful demonstrators viciousl...,2020-06-03 18:57:14.373699,Tue Jun 02 01:09:27 +0000 2020
7,1267543499722850309,Bernie Sanders,17902,This Pride Month we say: LGBTQ+ rights are civ...,2020-06-03 18:57:14.373699,Mon Jun 01 19:48:14 +0000 2020
8,1267505204133597184,Bernie Sanders,33049,"Liar in Chief: According to a new report, Trum...",2020-06-03 18:57:14.447191,Mon Jun 01 17:16:04 +0000 2020
9,1267475665479241728,Bernie Sanders,12483,Mitch McConnell says we should end the $600 un...,2020-06-03 18:57:14.447191,Mon Jun 01 15:18:41 +0000 2020


## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [7]:
# A:

### Look at the ngrams for Bernie Sanders

In [8]:
# A:

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [9]:
# A:

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [10]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####


## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [11]:
# A:

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [12]:
# A:

### 3. Make a confusion matrix and classification report.

In [13]:
# A:

### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [14]:
# A: